# importing the  libraries

In [7]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
train = pd.read_csv(r'C:\Users\Skyborn\train.csv')
test=pd.read_csv(r"C:\Users\Skyborn\test.csv")


In [9]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

Splitting out the X variable from the target

In [10]:
y = train['sentiment']
X = train['message']

In [11]:
from nltk.stem import PorterStemmer

porter_stemmer=PorterStemmer()

def my_cool_preprocessor(text):
    
    text=text.lower() 
    text=re.sub("\\W"," ",text)
    text=re.sub("\\s+(in|the|all|for|and|on)\\s+"," _connector_ ",text)
    
    words=re.split("\\s+",text)
    stemmed_words=[porter_stemmer.stem(word=word) for word in words]
    return ' '.join(stemmed_words)



In [12]:
def my_tokenizer(text):
    text=re.sub("(\\W)"," \\1 ",text)
    return re.split("\\s+",text)

# Turning text into something your model can read

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
X_vectorized = vectorizer.fit_transform(X)

# Splitting the training data into a training and validation set

In [14]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.3,shuffle=True, stratify=y, random_state=11)

# Training the model and evaluating using the validation set

In [15]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

# Checking the performance of our model on the validation set

In [16]:
f1_score(y_val, rfc_pred, average="macro")

0.5545218797710965

In [18]:
from sklearn import metrics

print(metrics.classification_report(y_val,rfc_pred))

              precision    recall  f1-score   support

          -1       0.85      0.21      0.34       389
           0       0.62      0.33      0.43       706
           1       0.68      0.92      0.78      2559
           2       0.80      0.58      0.67      1092

    accuracy                           0.70      4746
   macro avg       0.73      0.51      0.55      4746
weighted avg       0.71      0.70      0.67      4746



# getting our test set ready

In [21]:
testx = test['message']
test_vect = vectorizer.transform(testx)

# Making predictions on the test set and adding a sentiment column to our original test df

In [22]:
y_pred = rfc.predict(test_vect)

In [23]:
test['sentiment'] = y_pred

In [25]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


# Creating an output csv for submission

In [27]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)